In [1]:
!pip install selenium
!apt-get update
!apt install -yqq xvfb chromium-chromedriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 233 kB in 1s (159 kB/s)
Reading package lists... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.10).
0 upgraded, 0 newly i

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Khởi tạo các tùy chọn cho trình duyệt
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Khởi tạo trình duyệt Selenium
browser = webdriver.Chrome(options=chrome_options)

In [3]:
import datetime

current_month = datetime.datetime.now().strftime("%-m")
current_year = datetime.datetime.now().strftime("%Y")

# current_month = 6
# current_year = 2024

print(f"Current month: {current_month}")
print(f"Current year: {current_year}")


Current month: 5
Current year: 2024


In [4]:
# Mở trang web
browser.get(f'https://www.goodreads.com/book/popular_by_date/{current_year}/{current_month}')


In [5]:
wait = WebDriverWait(browser, 10)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Tạo một danh sách để lưu thông tin sách
books_info = []

# Lặp qua từng phần tử <article class="BookListItem">
for book_item in soup.find_all('article', class_='BookListItem'):
    # Lấy href của quyển sách
    book_link = book_item.find('a', class_='BookCard__clickCardTarget')['href']

    # Lấy tiêu đề của quyển sách
    title = book_item.find('h3', class_='Text__title3').text.strip()

    # Lấy contributor link
    author_link = book_item.find('a', class_='ContributorLink')['href']

    # Lấy tên của contributor và loại bỏ khoảng trắng thừa
    author_name = book_item.find('span', class_='ContributorLink__name').text.strip()
    author_name = ' '.join(author_name.split())

    # Thêm thông tin sách vào danh sách books_info
    books_info.append({
        'book_link': book_link,
        'title': title,
        'author_link': author_link,
        'author_name': author_name
    })

# In thông tin của tất cả các sách
for book_info in books_info:
    print("Url:", book_info['book_link'])
    print("Title:", book_info['title'])
    print("Author link:", book_info['author_link'])
    print("Author name:", book_info['author_name'])
    print()

Url: https://www.goodreads.com/book/show/197577126-this-summer-will-be-different
Title: This Summer Will Be Different
Author link: https://www.goodreads.com/author/show/21483807.Carley_Fortune
Author name: Carley Fortune

Url: https://www.goodreads.com/book/show/199797582-the-paradise-problem
Title: The Paradise Problem
Author link: https://www.goodreads.com/author/show/6556689.Christina_Lauren
Author name: Christina Lauren

Url: https://www.goodreads.com/book/show/136276877-the-last-murder-at-the-end-of-the-world
Title: The Last Murder at the End of the World
Author link: https://www.goodreads.com/author/show/17160667.Stuart_Turton
Author name: Stuart Turton

Url: https://www.goodreads.com/book/show/199798953-one-perfect-couple
Title: One Perfect Couple
Author link: https://www.goodreads.com/author/show/9013543.Ruth_Ware
Author name: Ruth Ware

Url: https://www.goodreads.com/book/show/195790670-if-something-happens-to-me
Title: If Something Happens to Me
Author link: https://www.goodr

In [7]:
# Tìm dòng chứa thông tin nextPageToken
next_page_token_script = soup.find('script', id='__NEXT_DATA__').text
# Tìm thông tin nextPageToken
import re
match = re.search(r'"nextPageToken":"([^"]+)"', next_page_token_script)
next_page_token = match.group(1)

print(next_page_token)

7B22746F704C6973744B6579223A2022626F6F6B732D62792D72656C656173652D646174652D323032342D355F415F414C4C222C202272616E6B223A20223135227D


In [8]:
import requests

url = "https://kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com/graphql"

payload = {
    "operationName": "getTopList",
    "variables": {
        "name": f"books-by-release-date-{current_year}-{current_month}",
        "period": "A",
        "location": "ALL",
        "limit": 185,
        "nextPageToken": f"{next_page_token}"
    },
    "query": "query getTopList($name: String!, $period: String!, $location: String!, $nextPageToken: String, $limit: Int) {\n  getTopList(\n    getTopListInput: {name: $name, period: $period, location: $location}\n    pagination: {after: $nextPageToken, limit: $limit}\n  ) {\n    name\n    period\n    location\n    pageInfo {\n      hasNextPage\n      nextPageToken\n      __typename\n    }\n    edges {\n      ... on TopListBookEdge {\n        rank\n        count\n        node {\n          ...BasicBookFragment\n          viewerShelving {\n            ...ViewerShelvingFragmentWTR\n            __typename\n          }\n          primaryContributorEdge {\n            ...BasicContributorFragment\n            __typename\n          }\n          secondaryContributorEdges {\n            ...BasicContributorFragment\n            __typename\n          }\n          work {\n            stats {\n              ratingsCount\n              textReviewsCount\n              averageRating\n              __typename\n            }\n            id\n            viewerShelvings {\n              ...ViewerShelvingFragmentWTR\n              __typename\n            }\n            __typename\n          }\n          ...ShelvedMultipleExclusiveFragment\n          __typename\n        }\n        __typename\n      }\n      ... on TopListUserEdge {\n        rank\n        count\n        node {\n          name\n          imageUrl\n          __typename\n        }\n        __typename\n      }\n      ... on TopListWorkEdge {\n        rank\n        count\n        node {\n          id\n          details {\n            bestBook {\n              titleComplete\n              ...BasicBookFragment\n              viewerShelving {\n                ...ViewerShelvingFragmentWTR\n                __typename\n              }\n              primaryContributorEdge {\n                ...BasicContributorFragment\n                __typename\n              }\n              secondaryContributorEdges {\n                ...BasicContributorFragment\n                __typename\n              }\n              work {\n                stats {\n                  ratingsCount\n                  textReviewsCount\n                  averageRating\n                  __typename\n                }\n                viewerShelvings {\n                  ...ViewerShelvingFragmentWTR\n                  __typename\n                }\n                __typename\n              }\n              ...ShelvedMultipleExclusiveFragment\n              __typename\n            }\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment BasicBookFragment on Book {\n  id\n  legacyId\n  imageUrl\n  description\n  title\n  titleComplete\n  webUrl\n  __typename\n}\n\nfragment BasicContributorFragment on BookContributorEdge {\n  node {\n    id\n    name\n    webUrl\n    isGrAuthor\n    __typename\n  }\n  role\n  __typename\n}\n\nfragment ViewerShelvingFragmentWTR on Shelving {\n  id\n  book {\n    id\n    __typename\n  }\n  shelf {\n    name\n    webUrl\n    __typename\n  }\n  taggings {\n    tag {\n      name\n      webUrl\n      __typename\n    }\n    __typename\n  }\n  webUrl\n  __typename\n}\n\nfragment ShelvedMultipleExclusiveFragment on Book {\n  reviewEditUrl\n  work {\n    id\n    editions {\n      webUrl\n      __typename\n    }\n    viewerShelvingsUrl\n    __typename\n  }\n  __typename\n}\n"
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'X-Api-Key': 'da2-xpgsdydkbregjhpr6ejzqdhuwy',
    'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, json=payload, timeout=10)
print(response)

if response.status_code == 200:
    data = response.json()
    print(data)
    books = data['data']['getTopList']['edges']

    for book in books:
        book_link = book['node']['webUrl']
        title = book['node']['title']
        author_link = book['node']['primaryContributorEdge']['node']['webUrl']
        author_name = book['node']['primaryContributorEdge']['node']['name']

        books_info.append({
            'url': url,
            'title': title,
            'author_link': author_link,
            'author_name': author_name
        })

        print("Book link:", book_link)
        print("Title:", title)
        print("Author link:", author_link)
        print("Author name:", author_name)
        print()

<Response [200]>
{'data': {'getTopList': {'name': 'books-by-release-date-2024-5', 'period': 'A', 'location': 'ALL', 'pageInfo': {'hasNextPage': False, 'nextPageToken': None, '__typename': 'PageInfo'}, 'edges': [{'rank': 16, 'count': 37349, 'node': {'id': 'kca://book/amzn1.gr.book.v3.NqP3K3vEOlU-nFMF', 'legacyId': 195790899, 'imageUrl': 'https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1695080579i/195790899.jpg', 'description': "Summer won't last forever.<br /><br />Sage Byrd has lived in the coastal town of Spunes, Oregon (not to be confused with Forks, Washington) her entire life. She's learned to love her small world, with the misfit animals on her hobby farm, and her friendships with the town’s inhabitants. But when her 5-year relationship ends and her ex, town-golden-boy Ian, suddenly gets engaged, Sage needs a win―something that will convince everyone to stop pitying her all the time, and to put Ian in his place. The Festival of Spunes, the tow

In [9]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PBL7/CrawlGoodread

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/PBL7/CrawlGoodread


In [10]:
import pandas as pd

df = pd.DataFrame(books_info)

df.to_csv('books_url.csv', index=False, encoding='utf-8-sig')